# Aula 4 - Solução dos exercícios
Leandro Carísio Fernandes

<br>

Enunciado do exercício da semana:

Treinar um modelo de linguagem em dados em português

- Avaliar o modelo usando a perplexidade, que é simplesmente a exponencial de todas as losses do dataset de validação
- Iremos treinar o modelo para prever o próximo token dado os anteriores (também conhecido como Causal Language Modeling). Não confundir com o Masked Language Modeling (MLM), que consiste em prever tokens mascarados em uma dada sequência (ex: BERT's MLM)

Dicas:

- Usar como ponto de partida o modelo OPT-125M, que já foi treinado em 300B de tokens (maioria em Inglês)
- Usar este dataset reduzido do mc4 portugues, com ~300M de tokens: gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt



## Instalação da biblioteca, configurações inicias e treinamento/criação do modelo

A célula abaixo contém as variáveis que podem ser configuradas nesse notebook:

In [1]:
# Indica se é pra recuperar o modelo do drive que já foi re-treinado com dados em português
# ou se é apenas para usar o modelo original opt-125m (False)
recuperar_modelo_do_drive = True

# Indica se é pra fazer um treinamento ou não, quantas épocas devem ser usadas e
# qual o % da base deve ser usado para treinamento (o resto é validação)
treinar_modelo = False
epochs = 3
porcentagem_dados_treinamento = 90 # Valor entre 0-100

# Se o treinamento for feito, indica se é pra salvar o modelo no drive ou não
salvar_modelo_no_drive = True

# Indica se é pra usar a amostra completa de 1GB
usar_sample_1gb = True

local_modelo_salvo_drive = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula4-modelo-linguagem-portugues/'


Monta o drive e verifica o tipo de GPU:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

Mounted at /content/drive
Wed Mar 29 16:38:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                              

Faz o download do arquivo que o professor passou, com 1GB de sample de texto em português:

In [3]:
!gsutil cp gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt .

# Ideia do caderno da Mirelle, pra pegar apenas 100 amostras e testar a execução do algoritmo completo
!sed -n '1,100p' sample-1gb.txt > small_sample.txt

Copying gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][  1.2 GiB/  1.2 GiB]                                                
Operation completed over 1 objects/1.2 GiB.                                      


Instala transformers e datasets:

In [4]:
!pip install transformers datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.4 MB/s eta 0:00:00


## Treinamento do modelo com dados em português

A ideia aqui é que o modelo é treinado e, depois, salvo no drive. Assim, depois podemos recuperar a versão que está no drive e treinar mais épocas (desde que mantendo o mesmo conjunto de treinamento):

In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

def carregar_modelo_e_tokenizer_do_drive():
  tokenizer = AutoTokenizer.from_pretrained(local_modelo_salvo_drive)
  model = AutoModelForCausalLM.from_pretrained(local_modelo_salvo_drive).to(device)
  return model, tokenizer

In [7]:
def carregar_modelo_opt_125_m():
  model_name = 'facebook/opt-125m'

  # Carregar o modelo pré-treinado
  model = AutoModelForCausalLM.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  return model, tokenizer

In [8]:
# Aqui decidimos se vamos carregar o modelo pré-treinado original ou se vamos carregar um que está salvo no drive
if recuperar_modelo_do_drive:
  model, tokenizer = carregar_modelo_e_tokenizer_do_drive()
else:
  model, tokenizer = carregar_modelo_opt_125_m()

In [9]:
%%time

from datasets import load_dataset

def treinar_modelo_com_sample_pt():
  data_file = './sample-1gb.txt' if usar_sample_1gb else './small_sample.txt'

  # Carregar o dataset em português
  #dataset = TextDataset(tokenizer=tokenizer, file_path=data_file, block_size=128)
  # Split copiado da Monique
  dataset_train = load_dataset("text", data_files=data_file, split=f"train[:{porcentagem_dados_treinamento}%]")
  tokenized_dataset_train = dataset_train.map(
      lambda x: tokenizer(x['text'], truncation=True, padding="max_length", max_length=128),
      batched=True,
      num_proc=4,
      remove_columns=["text"]
  )

  dataset_eval = load_dataset("text", data_files=data_file, split=f"train[{porcentagem_dados_treinamento}%:]")
  tokenized_dataset_eval = dataset_eval.map(
      lambda x: tokenizer(x['text'], truncation=True, padding="max_length", max_length=128),
      batched=True,
      num_proc=4,
      remove_columns=["text"]
  )

  # Definir o collator de treinamento
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

  # Definir os argumentos de treinamento
  training_args = TrainingArguments(
      output_dir=local_modelo_salvo_drive,
      num_train_epochs=epochs,
      per_device_train_batch_size = 16,
      per_device_eval_batch_size = 16,
      evaluation_strategy = "epoch",
      save_strategy="epoch",
      logging_strategy="epoch",
      learning_rate=2e-5,
      weight_decay=0.01,
      fp16=True
  )
           
  # Definir o trainer de treinamento
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset_train,
      eval_dataset=tokenized_dataset_eval,
      data_collator=data_collator,
  )

  # Treinar o modelo
  trainer.train()

  # Avaliar o modelo
  eval_results = trainer.evaluate()

  # Exibir a perplexidade
  perplexidade = torch.exp(torch.tensor(eval_results['eval_loss'])).item()
  print(f"\n**************************\nPerplexidade: {perplexidade:.2f}\n**************************\n")

  # Salva o modelo treinado e a tokenizer
  if salvar_modelo_no_drive:
    model.save_pretrained(local_modelo_salvo_drive)
    tokenizer.save_pretrained(local_modelo_salvo_drive)

def mostrar_perplexidade_modelo():
  data_file = './sample-1gb.txt' if usar_sample_1gb else './small_sample.txt'

  dataset_eval = load_dataset("text", data_files=data_file, split=f"train[{porcentagem_dados_treinamento}%:]")
  tokenized_dataset_eval = dataset_eval.map(
      lambda x: tokenizer(x['text'], truncation=True, padding="max_length", max_length=128),
      batched=True,
      num_proc=4,
      remove_columns=["text"]
  )

  # Definir o collator de treinamento
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
          
  # Definir o trainer de treinamento
  trainer = Trainer(
      model=model,
      args=None,
      train_dataset=None,
      eval_dataset=tokenized_dataset_eval,
      data_collator=data_collator,
  )

  # Avaliar o modelo
  eval_results = trainer.evaluate()

  # Exibir a perplexidade
  perplexidade = torch.exp(torch.tensor(eval_results['eval_loss'])).item()
  print(f"\n**************************\nPerplexidade: {perplexidade:.2f}\n**************************\n")

if treinar_modelo:
  treinar_modelo_com_sample_pt()
else:
  mostrar_perplexidade_modelo()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-27a34f089936a325/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



**************************
Perplexidade: 10.08
**************************

CPU times: user 5min 5s, sys: 11.5 s, total: 5min 16s
Wall time: 5min 11s


## Testes para prever a próxima palavra/frase

Foram feitos 3 testes aqui para gerar a próxima palavra:

1. Solicitei pro ChatGPT um código para isso. Esse código foi colocado na função preve_proxima_palavra(), mas não funcionou.

2. Uso do objeto pipeline.

3. Solicitei pro ChatGPT para "abrir" o código de pipeline.

In [20]:
from transformers import pipeline

# Teste de prever a próxima palavra - algoritmo básico gerado pelo Chatgpt (obs.: esse código não está ok)
def prever_proxima_palavra(frase, model, tokenizer, k):
  # Passo 1 - Tokenizar o texto de entrada usando o tokenizador do modelo. O tokenizador irá dividir o texto em uma sequência de tokens que podem ser usados como entrada para o modelo.
  input_ids = tokenizer.encode(frase, return_tensors='pt').to(device)

  # Passo 2 - Passar a sequência de tokens através do modelo para obter a distribuição de probabilidade condicional para o próximo token.
  with torch.no_grad():
    output = model(input_ids)
    next_token_logits = output.logits[:, -1, :]

  # Passo 3 - Obter a lista dos tokens com as maiores probabilidades condicionais.
  next_token_probs = torch.softmax(next_token_logits, dim=-1)
  top_next_token_probs, top_next_token_ids = torch.topk(next_token_probs, k=k)

  # Passo 4 - Converter os IDs dos tokens em texto usando o tokenizador e imprimir as palavras mais prováveis.
  retorno = []
  for i in range(top_next_token_ids.size(1)):
    next_token = tokenizer.decode(top_next_token_ids[0, i].item())
    retorno.append( (next_token, top_next_token_probs[0, i].item()) )

  return retorno

def continuar_frase_implementacao_chatgpt_1(frase, k=100):
  for i in range(k):
    palavras = prever_proxima_palavra(frase, model, tokenizer, 3)
    frase = frase + " " +  palavras[0][0]

  return frase


# Código gerado pelo ChatGPT quando solicitei para ele "abrir" a implementação do pipeline
# Única modificação foi ter extraído alguns parâmetros para a função
# temperatura, top_k e top_p são hiperparâmetros para a geração de texto
def continuar_frase_implementacao_chatgpt_2(frase, max_length=100, amostragem_estocastica=True, temperatura=0.7):
  # Tokeniza o texto inicial
  input_ids = tokenizer.encode(frase, return_tensors='pt').to(device)

  # Gera a sequência de texto
  output = model.generate(
      input_ids=input_ids,
      max_length=max_length,
      do_sample=amostragem_estocastica,
      temperature=temperatura
  )

  # Decodifica a sequência de texto gerada
  output_text = tokenizer.decode(output[0], skip_special_tokens=True)

  return output_text

def continuar_frase_pipeline(frase, max_length=100, amostragem_estocastica=True):
  device_generator = 0 if device.type == 'cuda' else None
  generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=device_generator)
  output = generator(frase, max_length=max_length, do_sample=amostragem_estocastica)
  return output[0]['generated_text']
  


In [21]:
import textwrap

def continuar_frase(frase, max_length=100, amostragem_estocastica=True, temperatura=0.7):
  print('Frase:\n "' + frase + '"\n')

  print('Primeira implementação do ChatGTP:')
  print(textwrap.fill(continuar_frase_implementacao_chatgpt_1(frase), 120))

  print('\nSegunda implementação do ChatGPT:')
  print(textwrap.fill(continuar_frase_implementacao_chatgpt_2(frase, max_length, amostragem_estocastica, temperatura), 120))

  print('\nChamando pipeline:')
  print(textwrap.fill(continuar_frase_pipeline(frase, max_length, amostragem_estocastica), 120))

  print('*'*120)

Exemplo de continuação da mesma frase com pontuação final diferente:

In [22]:
continuar_frase("Era uma vez", 150, False)
continuar_frase("Era uma vez.", 150, False)
continuar_frase("Era uma vez ", 150, False)
continuar_frase("Era uma vez  ", 150, False)

Frase:
 "Era uma vez"

Primeira implementação do ChatGTP:
Era uma vez  um  hom em  bus ca í  d as  m ais  e    e u ma is so  que    e    e    e    e    e    e    e    e    e    e
e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e    e
e    e    e    e    e    e    e

Segunda implementação do ChatGPT:
Era uma vez um homem que estava em uma caminhonete, quando foi atropelado por um caminhão. Ele foi atingido por um carro
e foi socorrido. O fato aconteceu na noite de ontem (14), no bairro São João, em Campo Grande. Segundo a Polícia
Militar, o homem estava em uma motocicleta e foi atropelado por um caminhão. Ele foi socorrido e foi até o Hospital
Regional de Campo Grande. O motorista do caminhão, que est

Chamando pipeline:
Era uma vez um homem que estava em uma caminhonete, quando foi atropelado por um caminhão. Ele foi atingido por um carro
e foi socorrido. O fato aconteceu na noite de ontem (14), no bairro São João, em Campo 

Alguns outros exemplos:

In [14]:
continuar_frase("O Tribunal de Contas da União", 150, False)

continuar_frase("Juro médio cobrado pelos bancos", 150, False)

Frase:
 "O Tribunal de Contas da União"

Primeira implementação do ChatGTP:
O Tribunal de Contas da União  ( TC U CS U  )  tem  por  final id ê  n  d as  se gu   ile g es  o u    e  o  p  a o  c
o  p  a  o    e  o    o    o    o    o    o    o    o    o    o    o    o    o    o    o    o    o    o    o    o    o
o    o    o    o    o    o    o    o    o    o    o    o

Segunda implementação do ChatGPT:
O Tribunal de Contas da União (TCU) determinou que o governo federal deve seguir a Lei de Responsabilidade Fiscal ( Lei
9.527/97) para que o Estado possa aprovar ações de controle e fiscalização de suas atividades. A decisão foi tomada na
última terça-feira (11/07). A Lei de Responsabilidade Fiscal (Lei 9.527/97) foi criada em 1999 e regulamentada pela Lei
de Responsabilidade Fiscal (Lei 9.826/97). A Lei de Responsabilidade

Chamando pipeline:
O Tribunal de Contas da União (TCU) determinou que o governo federal deve seguir a Lei de Responsabilidade Fiscal ( Lei
9.527/97) para que o Estad